## Preprocessing

In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import sklearn as skl
import pickle

#  Import and read the charity_data.csv.
import pandas as pd 
electionsdemographics_df = pd.read_csv("./Resources/clean_data.csv")
electionsdemographics_df.head()

,year,state,state_po,county_name,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,2012,MISSOURI,MO,LINN,12668.0,43.0,39028.0,20968.0,13.245974,Republican
1,2012,MISSOURI,MO,HOWELL,40330.0,39.6,34148.0,17763.0,20.300025,Republican
2,2012,MISSOURI,MO,JOHNSON,52964.0,29.7,47960.0,21375.0,15.246205,Republican
3,2012,MISSOURI,MO,LACLEDE,35507.0,39.1,39101.0,19788.0,18.244290,Republican
4,2012,MISSOURI,MO,MARIES,9140.0,43.1,44885.0,21883.0,14.070022,Republican


In [2]:
# Select data from the years we want to analyze
elec_dem_years_analysis_df = electionsdemographics_df[(electionsdemographics_df['year'] < 2020)]
elec_dem_years_analysis_df.head()

,year,state,state_po,county_name,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,2012,MISSOURI,MO,LINN,12668.0,43.0,39028.0,20968.0,13.245974,Republican
1,2012,MISSOURI,MO,HOWELL,40330.0,39.6,34148.0,17763.0,20.300025,Republican
2,2012,MISSOURI,MO,JOHNSON,52964.0,29.7,47960.0,21375.0,15.246205,Republican
3,2012,MISSOURI,MO,LACLEDE,35507.0,39.1,39101.0,19788.0,18.244290,Republican
4,2012,MISSOURI,MO,MARIES,9140.0,43.1,44885.0,21883.0,14.070022,Republican


In [3]:
# Drop the non-beneficial ID columns.
elec_dem_years_analysis_df = elec_dem_years_analysis_df.drop(['year','state','state_po','county_name'], axis = 1)
elec_dem_years_analysis_df.head()

,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,12668.0,43.0,39028.0,20968.0,13.245974,Republican
1,40330.0,39.6,34148.0,17763.0,20.300025,Republican
2,52964.0,29.7,47960.0,21375.0,15.246205,Republican
3,35507.0,39.1,39101.0,19788.0,18.244290,Republican
4,9140.0,43.1,44885.0,21883.0,14.070022,Republican


In [4]:
# Determine the number of unique values in each column.
column_unique_count = elec_dem_years_analysis_df.nunique()
column_unique_count

Population           5798
Median Age            330
Household Income     5494
Per Capita Income    5072
Poverty Rate         6002
Winner                  2
dtype: int64

In [5]:
# For columns with more than 2 unique values, determine the number of data points for each unique value
data_points = list(column_unique_count[column_unique_count>2].index)
print(data_points)

for x in data_points:
    print(elec_dem_years_analysis_df[x].value_counts())

['Population', 'Median Age', 'Household Income', 'Per Capita Income', 'Poverty Rate']
11526.0     3
16892.0     3
2095.0      3
14119.0     3
20324.0     3
           ..
5751.0      1
725.0       1
2753.0      1
7779.0      1
284559.0    1
Name: Population, Length: 5798, dtype: int64
41.2    71
40.4    69
40.3    65
40.2    64
40.8    62
        ..
53.7     1
24.6     1
54.9     1
25.1     1
57.4     1
Name: Median Age, Length: 330, dtype: int64
45000.0    4
38155.0    4
37750.0    4
43529.0    4
46071.0    4
          ..
49075.0    1
44426.0    1
39950.0    1
44106.0    1
46842.0    1
Name: Household Income, Length: 5494, dtype: int64
21716.0    4
24099.0    4
21696.0    4
21883.0    4
17504.0    4
          ..
22943.0    1
26062.0    1
22743.0    1
19421.0    1
26083.0    1
Name: Per Capita Income, Length: 5072, dtype: int64
7.142857     2
10.000000    2
16.000000    2
20.000000    2
13.245974    1
            ..
15.724931    1
19.808475    1
13.288922    1
23.166879    1
20.784793  

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
data_dummies = pd.get_dummies(elec_dem_years_analysis_df)
data_dummies.head()

,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner_Democrat,Winner_Republican
0,12668.0,43.0,39028.0,20968.0,13.245974,0,1
1,40330.0,39.6,34148.0,17763.0,20.300025,0,1
2,52964.0,29.7,47960.0,21375.0,15.246205,0,1
3,35507.0,39.1,39101.0,19788.0,18.244290,0,1
4,9140.0,43.1,44885.0,21883.0,14.070022,0,1


In [11]:
data_dummies_target = data_dummies.drop('Winner_Democrat', axis=1).rename(columns={"Winner_Republican": "Winner"})
data_dummies_target.head()

,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Winner
0,12668.0,43.0,39028.0,20968.0,13.245974,1
1,40330.0,39.6,34148.0,17763.0,20.300025,1
2,52964.0,29.7,47960.0,21375.0,15.246205,1
3,35507.0,39.1,39101.0,19788.0,18.244290,1
4,9140.0,43.1,44885.0,21883.0,14.070022,1


In [13]:
# Split our preprocessed data into our features and target arrays
X = data_dummies_target.drop('Winner', axis=1)
y = data_dummies_target['Winner']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
# Create a StandardScaler instances
X_scaler = skl.preprocessing.StandardScaler()

# Fit the StandardScaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=2, input_shape=(X_train_scaled.shape[1],)))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
        
    # Output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Compile the model
    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return nn_model


In [16]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [17]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 08s]
val_accuracy: 0.8661784529685974

Best val_accuracy So Far: 0.8701730966567993
Total elapsed time: 00h 03m 48s
INFO:tensorflow:Oracle triggered exit


In [18]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'num_layers': 4,
 'units_0': 3,
 'units_1': 1,
 'units_2': 9,
 'units_3': 5,
 'units_4': 9,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0042'}

In [19]:
# Evaluate the model using the test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

47/47 - 0s - loss: 0.3697 - accuracy: 0.8702 - 275ms/epoch - 6ms/step
Loss: 0.36967575550079346, Accuracy: 0.8701730966567993
